In [13]:
import numpy as np
import cvxpy as cp
import itertools
from itertools import product as iters

I. THE MYSTERY BOX

In [14]:
def P(a,b,x1,x2,y1,y2):
    if (a + b) % 2 == (x1*y1 + x2*y2) % 2:
        return 1/2
    else:
        return 0 

def Pa(a, x1,x2,y1,y2): 
    return P(a, 0, x1,x2,y1,y2) + P(a, 1, x1,x2,y1,y2)

def Pb(b, x1,x2,y1,y2): 
    return P(0, b, x1,x2,y1,y2) + P(1, b, x1,x2,y1,y2)

P1: Check NS condition

In [15]:
flag = True
for x1, y1, x2, y2 in iters(range(2), repeat=4): #range 2, repeat 4 for 4 variables
    for l1, l2 in iters(range(2), repeat=2):
        for a in range(0, 2):
            if Pa(a, x1, x2, 0, 0) != Pa(a, x1, x2, l1, l2):
                flag = False
        for b in range(0, 2):
            if Pb(b, 0, 0, y1, y2) != Pb(b, l1, l2, y1, y2):
                flag = False        
print("No-Signaling: ", flag)      

No-Signaling:  True


Confirmed that $P$ is a no-signalling distribution

P2: Is P local?

From the lecture notes : classical set, last page, solution Froissant

$$ \min \ \lambda$$
such that 
$$ P(\tilde{a}, \tilde{b}| x, y) + Q(\tilde{a}, \tilde{b}|x,y) = \sum_{a', b'} \mu(a', b') P_{a', b'}(\tilde{a}, \tilde{b}|x,y) $$

$$ \mu(a', b') \geq 0, \sum_{a', b'} \mu(a', b')  = 1$$

$$ \lambda \geq Q(\tilde{a}, \tilde{b}|x,y) \geq - \lambda$$

To be local $\lambda$ must be 0

In [16]:
def PL(a, b, x1, x2, y1, y2, μ):
    P = 0
    for a00, a01, a10, a11, b00, b01, b10, b11 in iters([0, 1], repeat = 8):
        ar = np.array([[a00, a01],[a10, a11]])
        br = np.array([[b00, b01],[b10, b11]])
        #Implemented deterministic (delta) distribution
        #flatten μ
        #μ = np.reshape(μ, (256, 1))
        #HERE WAS THE PROBLEM
        #WAS USING WRONG BINARY FUNCTION TO ADRESS THE CONTENTS
        indi = 1*a00+2*a01+4*a10+8*a11+16*b00+32*b01+64*b10+128*b11
        if ar[x1, x2] == a:
            if br[x1, x2] == b:
                P += μ[indi]
    return P

#Improve norm constraint
def norm(x1, x2, y1, y2, μ): #P_c is choice between distribution function. Do I even uset he local function anymore here?
    norm=0
    for a, b in iters([0, 1], repeat = 2):
        #this one makes more sense, still doesn't give me a proper result, but λ -> 0
        norm+= PL(a, b, x1, x2, y1, y2, μ)
        #incorrect:
        #indi = 1*a+2*b+4*x1+8*x2+16*y1+32*y2
        #norm+= μ[indi]
    return norm

In [17]:
λ = cp.Variable()
Q = cp.Variable((64,1)) # 64 rows, 1 column
P_ar = np.array([P(a, b, x1,x2,y1,y2) for a, b, x1,x2,y1,y2 in iters(range(0, 2), repeat=6 )])
P_ar = P_ar.reshape((64, 1)) #reshape because otherwise this shit shits itself
μ = cp.Variable(256) # 64 rows 4 columns 00, 01, 10, 11 

#my variant
Pain = cp.bmat([[PL(a, b, x1,x2,y1,y2, μ)] for a, b, x1,x2,y1,y2 in iters(range(0, 2), repeat=6 ) ])
#it needs the [] around the P to work , thanks David

"""Running the lp with constraints"""
constraints = [P_ar + Q == Pain]
constraints+= [μ >= 0] 

#Variant 1: bad
#constraints+= [cp.sum(μ) == 1] #this one is wrong

#Variant 2: Improved norm constraint
for x1, x2, y1, y2 in iters([0, 1], repeat=4):
    constraints+= [norm(x1, x2, y1, y2, μ) == 1]

#Is this right? I might need to write something more elaborate?
#maybe the problem is on how μ is applied on P
#the same 256 entries of μ are applied in each line during the cal. not sure that fits but w/e
#Would I need 64x256? Use a new μ for every line? Seems kinda overkill?
constraints+= [Q <= λ, Q >= -λ ]
prob = cp.Problem(cp.Minimize(λ),
                 constraints)
print(prob.solve())

0.25000000002487804


/home/pyr/anaconda3/envs/gp-env/lib/python3.8/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [18]:
#idk what I am doing wrong or what is missing
#comparing to David's, his μ constraints seem more elaborate
#todo: fix

#fixed the μ normalization
#still wrong
#well shit

Well this looks local, since λ is close to 0 i.e no noise needed to be added to make the distribution local

II. MAXIMIZING BELL FUNCTIONALS 

P.1 Prove equation (2)

Assume that $x$ is not an extreme point. Let there be an $v_m$ such that $f(v_m) = \max f(v_i) \forall i$, where $v_i$ are extreme points.

Since $x$ is not an extreme point it can be written as a convex combination of extreme points and using linearity we get:

$$ f(x) = f(\sum \lambda_i v_i) = \sum \lambda_i f(v_i) \leq \sum \lambda_i f(v_m) = f(v_m)$$

Obviously we use that $\sum \lambda_i = 1$

So for any point that can be written as a convex combination (i.e $\forall s \in S$) of extreme points there exists an extreme point that gives a higher value for the function i.e maximizes it.

P.2 Find the maximum of 

$$ B(P ) := P (0, 0, 0|0, 0, 0) + P (1, 1, 0|0, 1, 1) + P (0, 1, 1|1, 0, 1) + P (1, 0, 1|1, 1, 0) $$

for $P \in L$

Since $P \in L$ we know that extreme points maximize such a functional and that the extreme points are deterministic boxes of the form

$$ P(a, b, c|x, y, z) = \delta_{\tilde{a},a_x } \delta_{\tilde{b},b_y} \delta_{\tilde{c},c_z} $$

Using that and plugging in the values we get that 

$$\max (B(P)) = 1$$

P.3 Find maximum for $P \in NS$ numerically

The facets of the NS set correspond to the constraint $P(a,b|x,y) \geq 0 $

Maximize B(P) for $P \in NS$.

Use constraints: NS $P_a = \sum_a P$, and normalization.

Idea: Take the deterministic local function and add some noise to it under NS conditions to make a NS distribution. 

Try to maximize the Bell functional which depends on P, μ. 
Where μ will be added to P? 

In [19]:
#This is the deterministic function
#do I even need this?
def P_L(a, b, c, x, y, z, Q):
    Pl = 0
    for a0, a1, b0, b1, c0, c1 in iters([0, 1], repeat=6):
        ar = np.array([a0, a1])
        br = np.array([b0, b1])
        cr = np.array([c0, c1])
        if ar[x] == a:
            if br[y] == b:
                if cr[z] == c:
                    Pl += Q[1*a+2*b+4*c+8*x+16*y+32*z]
    return Pl

#So this is the distribution I that is used for optimizing
#this is the one I use to make my constraints
#for a given x,y,z the normalization is over a, b, c outputs
#So Q 2^6 = 64 entries, which we access with the binary expression
def P_ns(a, b, c ,x ,y, z, Q):
    s = Q[1*a+2*b+4*c+8*x+16*y+32*z]
    return s

#Define marginals
#Define them as P_a or P_ab? 
#eh I guess either way should be enough to satisfy NS
#reduce unnecessary args?

def P_La(a, x, y, z, P_c, Q):
    mgl = 0
    for k, l in iters([0,1], repeat=2):
        mgl+= P_c(a, k, l, x, y, z, Q)
    return mgl

def P_Lb(b, y, x, z, P_c, Q):
    mgl = 0
    for k, l in iters([0,1], repeat=2):
        mgl+= P_c(k, b, l, x, y, z, Q)
    return mgl

def P_Lc(c, z, x, y, P_c, Q):
    mgl = 0
    for k, l in iters([0,1], repeat=2):
        mgl+= P_c(k, l, c, x, y, z, Q)
    return mgl


#Bell functional
#We want to maximize this
def Bell(P, Q):
    val = P(0, 0, 0, 0, 0, 0, Q)+P(1, 1, 0, 0, 1, 1, Q) +P(0, 1, 1, 1, 0, 1, Q)+P(1, 0, 1, 1, 1, 0, Q)
    return val


#Set up conditions

#NS conditions for the distribution
#Based on the marginals defined above
def NS_cond(P_c, Q):
    #P takes 6 args
    condi = []
    for dist in [P_La, P_Lb, P_Lc]:
        for var, l1, l2, l3 in iters([0,1], repeat = 4):
            condi.append( dist(var, l1, l2, l3, P_c, Q) == dist(var, l1, 0, 0, P_c, Q) )
    return condi

#Normalization of Q
#for a given x,y,z output over all a, b, c must be normalized
def norm(x, y, z, P_c, Q): #P_c is choice between distribution function. Do I even uset he local function anymore here?
    norm=0
    for a, b, c, in iters([0, 1], repeat = 3):
        norm+=P_c(a, b, c, x, y, z, Q) 
    return norm
    #i could have done all the iterations in here but w/e

In [20]:
#maybe useless
def P_Lab(a, b, c, x, y, z, μ):
    mgl = 0
    for l in range(0, 2):
        mgl+= P_L(a, b, l, x, y, z, μ)
    return mgl

def P_Lac(a, b, c, x, y, z, μ):
    mgl = 0
    for l in range(0, 2):
        mgl+= P_L(a, l, c, x, y, z, μ)
    return mgl

def P_Lbc(a, b, c, x, y, z, μ):
    mgl = 0
    for l in range(0, 2):
        mgl+= P_L(l, b, c, x, y, z, μ)
    return mgl

In [21]:
Q = cp.Variable(64)
condi = [Q >= 0]
condi+= NS_cond(P_ns, Q)
for x, y, z in iters([0, 1], repeat=3):
    condi+= [norm(x, y, z, P_ns, Q) == 1]

In [22]:
problem = cp.Problem(cp.Maximize(Bell(P_ns, Q)), condi)
maximum_NS = problem.solve()

In [23]:
maximum_NS

1.999999997125511

Result is 1.9999 something so close to 2. Seems okay.

In [24]:
#I did obtain the right result with the wrong code previously though.
#But now I think it should be fine